# Multiple Choice

In [1]:
import pandas as pd
import re
from tqdm.auto import tqdm
from vllm import LLM, SamplingParams
import torch

# ======== CONFIG =====================
CSV_PATH = "/project/ai901503-ai0003/Top/Financial_Analysis_Agent/test.csv"
MODEL_PATH = "/project/ai901503-ai0003/Model/Qwen3-30B-A3B"
SUB_PATH = "/project/ai901503-ai0003/Top/Financial_Analysis_Agent/submission.csv"
SAVE_PATH = "/project/ai901503-ai0003/Top/Financial_Analysis_Agent/Qwen3-30B-A3-English_Prompt-Nonthink-Final.csv"
MAX_RETRY = 3
REASONING = False  # ✅ Toggle reasoning

# ======== LOAD LLM ===================
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    min_p=0,
    max_tokens=32768
)

INFO 06-26 05:04:19 [__init__.py:244] Automatically detected platform cuda.


In [2]:
llm = LLM(
    model=MODEL_PATH,
    dtype=torch.float16,
    tensor_parallel_size=4,
    task="generate"
)

WARNING 06-26 05:04:32 [config.py:3271] Casting torch.bfloat16 to torch.float16.
INFO 06-26 05:04:32 [config.py:1946] Defaulting to use mp for distributed inference
INFO 06-26 05:04:32 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-26 05:04:34 [core.py:455] Waiting for init message from front-end.
INFO 06-26 05:04:34 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='/project/ai901503-ai0003/Model/Qwen3-30B-A3B', speculative_config=None, tokenizer='/project/ai901503-ai0003/Model/Qwen3-30B-A3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=40960, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', d

Loading safetensors checkpoint shards:   0% Completed | 0/16 [00:00<?, ?it/s]


(VllmWorker rank=3 pid=67590) INFO 06-26 05:07:31 [default_loader.py:272] Loading weights took 172.86 seconds
(VllmWorker rank=2 pid=67589) INFO 06-26 05:07:31 [default_loader.py:272] Loading weights took 172.88 seconds
(VllmWorker rank=0 pid=67587) INFO 06-26 05:07:31 [default_loader.py:272] Loading weights took 172.88 seconds
(VllmWorker rank=1 pid=67588) INFO 06-26 05:07:31 [default_loader.py:272] Loading weights took 172.90 seconds
(VllmWorker rank=3 pid=67590) INFO 06-26 05:07:32 [gpu_model_runner.py:1624] Model loading took 14.2479 GiB and 173.811249 seconds
(VllmWorker rank=2 pid=67589) INFO 06-26 05:07:32 [gpu_model_runner.py:1624] Model loading took 14.2479 GiB and 173.846326 seconds
(VllmWorker rank=0 pid=67587) INFO 06-26 05:07:32 [gpu_model_runner.py:1624] Model loading took 14.2479 GiB and 173.776258 seconds
(VllmWorker rank=1 pid=67588) INFO 06-26 05:07:32 [gpu_model_runner.py:1624] Model loading took 14.2479 GiB and 173.871563 seconds
(VllmWorker rank=3 pid=67590) INFO 0

In [3]:
# ======== PROMPTS ====================
def get_system_prompt(reasoning: bool = True):
    if reasoning:
        return """You are an exceptional financial analyst competing in Mr. Beast's ultimate challenge.
        Carefully read the following question, which may be in Chinese or English, and respond with only one word. Your answer must be one of the following six options:
        - A
        - B
        - C
        - D
        - Rise
        - Fall

        Instructions:
        - If it's a multiple-choice question, answer with A, B, C, or D.
        - If it's a trend-related question (e.g., market or price movements), answer with Rise or Fall.
        - Do not explain your answer.
        - Do not use any other words or symbols.
        - Apply your sharpest reasoning—you're competing for a prize of over 100 million RMB"""
    else:
        return """You are an exceptional financial analyst competing in Mr. Beast's ultimate challenge.
        Carefully read the following question, which may be in Chinese or English, and respond with only one word. Your answer must be one of the following six options:
        - A
        - B
        - C
        - D
        - Rise
        - Fall

        Instructions:
        - If it's a multiple-choice question, answer with A, B, C, or D.
        - If it's a trend-related question (e.g., market or price movements), answer with Rise or Fall.
        - Do not explain your answer.
        - Do not use any other words or symbols.
        - Apply your sharpest reasoning—you're competing for a prize of over 100 million RMB"""

def get_judge_prompt():
    return """You are a strict but fair evaluator in a financial analysis competition.

    Your task is to determine if the given answer is correct.

    Instructions:
    1. Analyze whether the answer is correct based on the context or financial logic.
    2. Provide your reasoning inside <WHY> ... </WHY> — **do not repeat the question**.
    3. At the end, output <FINAL>TRUE</FINAL> if the answer is correct, or <FINAL>FALSE</FINAL> if it is incorrect.

    Guidelines:
    - If it's a multiple-choice question (A, B, C, D), judge the logic of the selected option based on known knowledge or hints.
    - If it's a trend question (e.g., Rise/Fall), use financial reasoning or context cues to decide if the prediction is likely accurate.
    - If the answer seems invalid, irrelevant, or lacks justification, mark it as incorrect.
    - Your explanation should be clear enough to guide model correction if needed.

    Format:

    <WHY>
    {Explain here why the answer is correct or incorrect without repeating the question}
    </WHY>
    <FINAL>{TRUE or FALSE}</FINAL>
    """

def estimate_tokens(text):
    latin = all(ord(c) < 128 for c in text)
    return int(len(text.split()) * (1.3 if latin else 2.5))

# ======== RUN LOOP =====================
df = pd.read_csv(CSV_PATH)
sub = pd.read_csv(SUB_PATH)
# df = df.head(5)
# sub = sub.head(5)

answers = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Inferencing"):
    question = row["query"]
    system_prompt = get_system_prompt(REASONING)

    # print(f"\n🟨 Question: {question}")

    for attempt in range(MAX_RETRY):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question},
        ]
        output = llm.chat(
            messages,
            sampling_params=sampling_params,
            chat_template_kwargs={"enable_thinking": REASONING}
        )
        text_output = output[0].outputs[0].text.strip()
        # print(f"📤 Raw output: {text_output}")

        # === Extract answer ===
        if not REASONING:
            match = re.search(r"<FINAL>(.*?)</FINAL>", text_output)
            answer = match.group(1).strip() if match else "?"
        else:
            match = re.search(r"\b(A|B|C|D|Rise|Fall|เพิ่มขึ้น|ลง)\b", text_output)
            answer = match.group(1) if match else "?"

        # Normalize Thai
        if answer == "เพิ่มขึ้น": answer = "Rise"
        elif answer == "ลง": answer = "Fall"

        # ✅ Use previous answer if current is "?" and not first attempt
        if answer == "?" and attempt > 0:
            answer = answers[-1] if answers else "?"

        # print(f"✅ Final Answer: {answer}")

        # ==== Judge step ====
        judge_prompt = get_judge_prompt()
        judge_msg = [
            {"role": "system", "content": judge_prompt},
            {"role": "user", "content": f"<QUESTION>{question}</QUESTION><ANSWER>{answer}</ANSWER>"}
        ]
        token_est = estimate_tokens(question + answer)
        if token_est > 150:
            # print("⚠️ Judge skipped due to token limit.")
            break  # accept answer
        judge_out = llm.chat(judge_msg, sampling_params)[0].outputs[0].text
        is_correct = "TRUE" in judge_out
        why = re.search(r"<WHY>(.*?)</WHY>", judge_out, re.DOTALL)
        feedback = why.group(1).strip() if why else ""

        # print(f"👨‍⚖️ Judge: {'✅ Correct' if is_correct else '❌ Incorrect'}")
        # if feedback:
        #     print(f"📝 Judge reason: {feedback}")

        if is_correct:
            break  # exit retry loop
        question += f"\n\n#Feedback: {feedback}"

    answers.append(text_output)

df["response"] = answers

Inferencing:   0%|          | 0/499 [00:00<?, ?it/s]

INFO 06-26 05:08:30 [chat_utils.py:420] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [4]:
# === OPTIONAL: merge submission format ===
sub = sub.merge(df[["id", "response"]], on="id", how="left")
final_sub = sub[["id", "response"]].rename(columns={"response": "answer"})

final_sub.to_csv(SAVE_PATH, index=False)
print(f"\n✅ Done! Saved to {SAVE_PATH}")


✅ Done! Saved to /project/ai901503-ai0003/Top/Financial_Analysis_Agent/Qwen3-30B-A3-English_Prompt-Nonthink-Final.csv


# Rise/Fall

In [5]:
import pandas as pd
import numpy as np
from helper import *
from transformers import AutoTokenizer

In [6]:
tokenizer_name = '/project/ai901503-ai0003/Model/typhoon21-gemma3-12b'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [7]:
def model_generate(question):
    system_prompt = '''
        ###Role
        You are a financial time series expert. 

        ###Instructions
        The following is a summary of engineered features and exploratory data analysis from a stock price dataset. 
        Use this information to reason whether the price is likely to rise or fall in the next 5 days.

        ###Details
        - Rise is when the price increases after 5 days.
        - Fall is when the price decreases after 5 days.

        ###Output
        1. **Always** give short reasons why the price is likely to rise or fall.
        2. Use concise language.
        3. Answer with a single choice: "Rise" or "Fall". **ONLY use these two options**
        4. Your final answer must be wrapped **exactly** in: <answer>YourAnswer</answer> **THIS IS VERY IMPORTANT**


        '''
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question + " Let's think step by step and answer the question."},
    ]
    
    outputs = llm.chat(messages, sampling_params, chat_template_kwargs={"enable_thinking": False})
    token = len(tokenizer.encode(outputs[0].outputs[0].text, add_special_tokens=False))
    print(f"Token length: {token}")
    print(outputs[0].outputs[0].text)
    
    match = re.search(r'<answer>(.*?)</answer>', outputs[0].outputs[0].text)

    if match:
        result = match.group(1)
        print(result)
        return result
    else:
        print(np.nan)
        return np.nan

In [8]:
df = pd.read_csv(CSV_PATH)
sub = pd.read_csv(SUB_PATH)

stock_df = get_stock_df(df)
stock_submission = stock_df.copy()
stock_submission['answer'] = np.nan
stock_submission = stock_submission.reset_index(drop=True)

for n, question in enumerate(tqdm(stock_df['query'])):
    prompt = cal_stat(question)
    # print(prompt)
    output = model_generate(prompt)
 
    stock_submission.loc[n, "answer"] = output
    
stock_submission.to_csv("/project/ai901503-ai0003/Top/Financial_Analysis_Agent/stock_submission.csv", index=False)

  0%|          | 0/101 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Token length: 101
Based on the feature averages, "Rise" days have negative momentum and lower trend_up values, while "Fall" days have positive momentum and higher trend_up values. However, the correlation shows that momentum and trend_up are strongly negatively correlated with the target. This suggests that lower momentum and trend_up are associated with a rise. 

If the new sample has low momentum and trend_up, it is more likely to be a "Rise".

<answer>Rise</answer>
Rise


/tmp/ipykernel_63528/961132610.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Rise' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  stock_submission.loc[n, "answer"] = output


...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Token length: 61
Based on the feature averages, "Rise" days have negative momentum and a trend_up of 0, while "Fall" days have positive momentum and a trend_up of 0.667. This suggests a higher likelihood of a fall. 

<answer>Fall</answer>
Fall


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Token length: 98
Based on the feature averages by class, "Fall" days have a trend_up of 0.5, indicating a 50% chance of closing above the open, but also have near-zero or negative momentum. Since there are no "Rise" samples in the target distribution, the model has no evidence of upward movement. The lack of significant correlation and the dominance of "Fall" days suggest a higher likelihood of a price fall.

<answer>Fall</answer>
Fall


In [9]:
submission = pd.read_csv(SAVE_PATH)
final_submission =  submission.merge(stock_submission, on="id", how = 'left')
final_submission['answer_y'] = final_submission['answer_y'].fillna(final_submission['answer_x'])
super_final_submission = final_submission[['id', 'answer_y']].rename(columns={'answer_y': 'answer'})

super_final_submission.to_csv("/project/ai901503-ai0003/Top/Financial_Analysis_Agent/Final_Submission.csv", index=False)